In [ ]:
# Anchana 2.0

# getting all functions to another sheet
# adding clustering logic
# adding anamoly detection / novelty detection algorithms

In [1]:
# Libs



# data structures
import pandas as pd
import numpy as np

# zSrv
# my custom functions
from ipynb.fs.full.Anchana_version2_functions import numvis_pipe, preproc, evalR, evalC, evalG, run_pca
# my folder path
from pathlib import Path
myfolder = Path("C:/Users/pdpc/Documents/MLfiles/Data/")

# graphs
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
%matplotlib inline

# train/test split
from sklearn.model_selection import train_test_split

# supress warning
import warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

# time function for timing program run
import time

## patches for overcoming
np.float = float    
np.int = int   #module 'numpy' has no attribute 'int'
np.object = object    #module 'numpy' has no attribute 'object'
np.bool = bool    #module 'numpy' has no attribute 'bool'

In [9]:
# Data

breastcancer = pd.read_excel(myfolder / "breastcancer.xlsx", sheet_name="breastcancer_unchanged")

breastcancer.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [10]:
breastcancer.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [13]:
fig = px.histogram(breastcancer, x="radius_mean",color='diagnosis')

fig.show()

<IPython.core.display.Javascript object>

In [20]:
fig = px.histogram(breastcancer, x="concavity_mean",color='diagnosis')

fig.show()

<IPython.core.display.Javascript object>

In [5]:
# Data

lung = pd.read_excel(myfolder / "cancer lung cancer.xlsx", sheet_name="cancer lung cancer")

lung.describe()

,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN
count,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000
mean,62.673139,1.563107,1.569579,1.498382,1.501618,1.504854,1.673139,1.556634,1.556634,1.556634,1.579288,1.640777,1.469256,1.556634
std,8.210301,0.496806,0.495938,0.500808,0.500808,0.500787,0.469827,0.497588,0.497588,0.497588,0.494474,0.480551,0.499863,0.497588
min,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,57.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,62.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000
75%,69.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,87.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [6]:
lung.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO


In [8]:
fig = px.histogram(lung, x="SMOKING",color='LUNG_CANCER')

fig.show()

<IPython.core.display.Javascript object>

In [4]:
# Data

data = pd.read_excel(myfolder / "cancer prostrate cancer.xlsx", sheet_name="cancer prostrate cancer")

data.describe()

,id,radius,texture,perimeter,area,smoothness,compactness,symmetry,fractal_dimension
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,50.500000,16.850000,18.230000,96.780000,702.880000,0.102730,0.126700,0.193170,0.064690
std,29.011492,4.879094,5.192954,23.676089,319.710895,0.014642,0.061144,0.030785,0.008151
min,1.000000,9.000000,11.000000,52.000000,202.000000,0.070000,0.038000,0.135000,0.053000
25%,25.750000,12.000000,14.000000,82.500000,476.750000,0.093500,0.080500,0.172000,0.059000
50%,50.500000,17.000000,17.500000,94.000000,644.000000,0.102000,0.118500,0.190000,0.063000
75%,75.250000,21.000000,22.250000,114.250000,917.000000,0.112000,0.157000,0.209000,0.069000
max,100.000000,25.000000,27.000000,172.000000,1878.000000,0.143000,0.345000,0.304000,0.097000


In [58]:
# Required User Input on Data

datasetname = 'CreditCardFraud'

drop_col = ['']

ProblemType = 'C' # 'R' for regression // 'C' for classifier // 'G' for clustering

n = 2 # for number of clusers

y_column = ["Class"]

In [63]:
# one-stop-execution for R and C

# Drop unneeded column(s) by column name(s)
data_small = data_small.drop(drop_col, axis=1, errors='ignore')

# X/y split
X  = data_small.drop(y_column, axis=1, errors='ignore')
y  = data_small[y_column].copy()

# Preprocess
X_preproc = preproc(X)

In [64]:
numvis_pipe(X)

Dataset shape:   (31200, 30)
Dataset size:    936000
Duplicate rows:  27
Num features:    30
Cat features:    0
Null features:   0
# of features with normal distribution: 0


In [66]:
pca = PCA(n_components=0.95)

X_featsel = pca.fit_transform(X_preproc)

pca.explained_variance_ratio_

a = np.sum(pca.explained_variance_ratio_) * 100
print("Preserved variance is:", "%.2f" % a,"%","\n")
#print("PCA features:",pca.explained_variance_ratio_)
print("number of original features:",X.shape[1])
print("number of new features:",pca.n_components_)

Preserved variance is: 95.66 % 

number of original features: 30
number of new features: 16


In [67]:

tic = time.time()

if ProblemType == 'R':
    D = evalR(X_preproc, y)
    A = 'Regression Models'
elif ProblemType == 'C':
    D = evalC(X_featsel, y)
    A = 'Classifier Models'
elif ProblemType == 'G':
    D = evalG(X_preproc, n)
    A = 'Clustering Models'

P2 = "Quick analysis of your dataset " + datasetname + ' using ' + A
print(P2)

toc = time.time()
print('time taken for evaluation:', round(toc-tic) , 'seconds' , "\n")

D

Quick analysis of your dataset CreditCardFraud using Classifier Models
time taken for evaluation: 38 seconds 



,LOG,KNN,GBC,SVC,DEC,RAF,ABC,ETC,MLP,LDA
Acc_Train,0.9986,0.9994,0.9990,0.9997,1.0000,1.0000,1.0000,1.0000,0.9995,0.9993
Acc_Test,0.9986,0.9989,0.9984,0.9992,0.9986,0.9990,0.9982,0.9995,0.9990,0.9986


In [19]:
    import numpy as np
    import matplotlib.pyplot as plt
    
    # PCA
    from sklearn.decomposition import PCA
    # Models
    from sklearn.cluster import KMeans, AgglomerativeClustering
    # Colors
    import matplotlib.colors as mcolors
    mcolors.cbook
    
    # Kmeans Clustering
    # Define model
    reg_kmn = KMeans(n_clusters = n, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    # fit & predict
    y_pred_kmn = reg_kmn.fit_predict(X_preproc)
    
    # Hierarchical Agglomerative Clustering
    # Define model
    reg_ahc = AgglomerativeClustering(n_clusters = n, affinity = 'euclidean', linkage = 'ward')
    # fit & predict
    y_pred_ahc = reg_ahc.fit_predict(X_preproc)
 
    # Visualizing clustering
    #1 visualizing original data
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_preproc)
    a = np.sum(pca.explained_variance_ratio_) * 100

    #plot
    plt.figure(figsize=(8,4))
    plt.title('Original Data in 2D')
    plt.xlabel('First principle component')
    plt.ylabel('Second principle component')
    plt.scatter(X_pca[:,0],X_pca[:,1])

    #2 visualizing Kmeans clustering

    # convert kmeans cluster centers to 2 dimensional axes
    A = reg_kmn.cluster_centers_
    pca = PCA(n_components=2)
    B = pca.fit_transform(A)

    # Plot iterable, coloring palette and datapoint size
    a = list(range(0, reg_kmn.n_clusters)) # number of iterations for for-loop = n_clusters
    c = ['green','cyan','yellow','red','magenta', 'blue','purple','black','white'] # setting colors
    s = 20 # setting datapoint size

    #plot
    plt.figure(figsize=(8,4))
    plt.title('Kmeans cluster')
    plt.xlabel('First principle component')
    plt.ylabel('Second principle component')
    for a,c in zip(a,c):
        plt.scatter(X_pca[y_pred_kmn==a,0], X_pca[y_pred_kmn==a,1], s = s, c=c)
    #Plotting the centroids of the clusters
    plt.scatter(B[:,0],B[:,1],s = 150, color='purple',marker='*',label='centroid')
    
    #3 visualizing Agglomerative Hierarchical clustering

    # Plot iterable, coloring palette and datapoint size
    a = list(range(0, reg_kmn.n_clusters)) # number of iterations for for-loop = n_clusters
    c = ['green','cyan','yellow','red','magenta', 'blue','purple','black','white'] # setting colors
    s = 20 # setting datapoint size

    #plot
    plt.figure(figsize=(8,4))
    plt.title('Agglomerative Hierarchical cluster')
    plt.xlabel('First principle component')
    plt.ylabel('Second principle component')
    plt.annotate(text = data["Country"])
    for a,c in zip(a,c):
        plt.scatter(X_pca[y_pred_ahc==a,0], X_pca[y_pred_ahc==a,1], s = s, c=c)
    
    # Evaluation
    # Evaluating clustering models is tricky since the process has NO truth labels
    # some metrics - Silhouette Coefficient / Dunn's Index - explore more

MemoryError: Unable to allocate 90.7 GiB for an array with shape (12167610003,) and data type float64

In [13]:
data["Country"]

0              China
1                USA
2              India
3             Brazil
4             Russia
5          Indonesia
6             Mexico
7              Japan
8            Germany
9            Nigeria
10            Canada
11          Pakistan
12            France
13              Iran
14    United Kingdom
15            Turkey
16             Italy
17        Bangladesh
18         Australia
19             Egypt
20         Argentina
21       Philippines
22      Saudi Arabia
23          Thailand
24          Ethiopia
25         DRC Congo
26             Spain
27       South Korea
28      South Africa
29           Vietnam
30          Colombia
31           Algeria
32             Sudan
33              Peru
34        Kazakhstan
35            Angola
36       Netherlands
37       Switzerland
38              Mali
39             Libya
40             Niger
41              Chad
42          Mongolia
Name: Country, dtype: object

In [62]:
# Minidata - sampling

# n - number of samples, frac - fraction of the original data to retain 
data_small = data.sample(frac=0.2)
#data_smaller = data_small.sample(frac=0.1)


#data = data.sample(n=2, weights='num_specimen_seen')

          
          

In [8]:
# Important Features based on Random Forest Regressor/Classifier GINI importance

A = RandomForestRegressor().fit(X_train,y_train).feature_importances_
Imp_fea = pd.DataFrame(A).T
Imp_fea.columns = X.columns
Imp_fea = Imp_fea.T
Imp_fea.columns = ['Imp Features']
Imp_fea.sort_values(by=['Imp Features'],ascending=False)

<IPython.core.display.Javascript object>

ValueError: could not convert string to float: 'F'